In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure
from skimage.filters import threshold_yen, threshold_isodata
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
from pims import pipeline, Video
from glob import glob
import seaborn as sns
#import antigravity

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number
def total_threshold_filter(frame, frame_no, transition_threshold, 
                           broken_count, last_class, last_whole, filtrate_len, 
                           sides, classes, transitions): 
    m_lab = measure.label
    m_rop = measure.regionprops
    # returns a new list, different from the orginal
    thresh_img = frame > threshold_yen(frame) # binary image
    img_labelled = m_lab(thresh_img) # contains connected regions
    properties_list = m_rop(img_labelled, coordinates = 'rc') # data about regions, for each connected region

    #----------getting maximum connected region----------
    biggest_r = max(properties_list, key = attrgetter('area'))
    """ for k in range(len(properties_list)):
        if areas[k] > biggest_area:
            biggest_r = properties_list[k]
        else:
            pass"""

    #----------------------filter------------------------- can use actual filter
    test = (biggest_r.filled_area > area_threshold and   # must have enough particles
        biggest_r.minor_axis_length > minor_thresholds[0] # minor_axis length (< filterd) # particles must be in parallelogram
        and biggest_r.minor_axis_length < minor_thresholds[1]  # minor_axis length (< filterd) # particles must be in parallelogram
        and biggest_r.convex_area > convex_thresholds[0]  # convex hull area (> 2500 filtered)  # particles must be in parallelogram
        and biggest_r.convex_area < convex_thresholds[1]  # convex hull area (> 2500 filtered)  # particles must be in parallelogram
        and biggest_r.major_axis_length > major_thresholds[0]  # major_axis length (< filterd) # particles must be in parallelogram
        and biggest_r.major_axis_length < major_thresholds[1])  # major_axis length (< filterd) # particles must be in parallelogram
    if test: # keep frames that have enough paricles, and are not transitions
        i_threshed = frame > threshold_isodata(frame)
        img2_labelled = m_lab(i_threshed)
        properties2_list = m_rop(img2_labelled, coordinates = 'rc')
        biggest_r2 = max(properties2_list, key = attrgetter('area'))
        #-----------------------------classify pt 1---------------------------------------
        coprod = coord_prod(thresh_img, i_threshed)
        side = sideify(biggest_r2)
        clas = classify(side, -coprod)
        sides[side].append(frame_no)
        classes[clas].append(frame_no)
        if broken_count >= transition_threshold and side != 'ucf': #if previous n frames were broken, and a transition between identifiableclasses ocurred
            transitions["->" + clas].append((last_whole, frame_no))
        broken_count = 0
        last_class = clas
        last_whole = frame_no
        filtrate_len += 1
    else:
        broken_count += 1

    return [broken_count, last_class, last_whole, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify(idir):
    iner = idir.inertia_tensor[0, 0] + idir.inertia_tensor[1, 1]
    maal = idir.major_axis_length
    mial = idir.minor_axis_length
    sol = idir.solidity 
    ecc = idir.eccentricity
    cva = idir.convex_area
    if sol <= 0.725 or (iner> 405 and mial > 43.5): # sol <.72
        return "bm"
    elif ecc > 0.8 or maal > 65: # or (mial < 41 and maal > 65): # it's on top #ec >.82
        return "t"
    elif ecc < 0.75 or maal < 64.5: #or mial > 42  #and cva > 2200 and iner < 325: #or (maal < 64 and iner < 360): # it's on bottom
        return "b"
    else:
        return "ucf"

def classify(side, coprod):
    if side == "t":
        if coprod > 0:
            return "tl"
        else:
            return "tr"
    elif side == "b": 
        if coprod > 0:
            return "br"
        else:
            return "bl"
    elif side == "bm":
        return "bm"
    elif side == "ucf":
        return "ucf"
    else:
        return "newt"


def coord_prod(thresh_y, thresh_i):
    m_rop = measure.regionprops
    for i in range(0, 1):#len(fl):
        full = largest_region_extractor(measure.label(thresh_y))
        four = largest_region_extractor(measure.label(thresh_i))
        full_or = m_rop(full.astype(int), coordinates = 'rc')[0].orientation
        one_and_some = full^four
        one = largest_region_extractor(measure.label(one_and_some))
        
          
        #Begin finding substitue for major and minor axes herre
        # use min and maax x and  values to find the geometric center
        # or use those to substitue for eigenvectors 
        rotated_one = ndimage.rotate(one, math.degrees(-full_or), reshape = False)
        rotated_full = ndimage.rotate(full, math.degrees(-full_or), reshape = False)
        geo_cent = m_rop(rotated_full.astype(int), coordinates = 'rc')[0].bbox
        #print(geo_cent)
        ave = np.average
        cent_x = ave([geo_cent[1], geo_cent[3]]) 
        cent_y = ave([geo_cent[0], geo_cent[2]])
        #print(cent_x, cent_y)
        
        raw_coords = (m_rop(rotated_full.astype(int), coordinates = 'rc')[0].coords).T
        ys = raw_coords[0]
        xs = raw_coords[1]
        x = xs - cent_x
        y = ys - cent_y      
        
        one_com = m_rop(rotated_one.astype(int))[0].centroid
        
        rel_com_one = (one_com[1] - cent_x, one_com[0] - cent_y)
        product = np.prod(rel_com_one)
        return product
        
def rotate_point(point, angle):
    x0 = point[0]
    y0 = point[1]
    x1 = x0*np.cos(angle) - y0*np.sin(angle)
    y1 = x0*np.sin(angle) + y0*np.cos(angle)
    return (x1, y1)
 
def region_selector(labeled_image, label):
    #print(label)
    x = labeled_image == label
    #plt.imshow(x)
    return x

# make a function that takes a set of labeled regions, and then  returns a boolean array containing only the largest
def largest_region_extractor(labeled_regions_set):
    props_lists = measure.regionprops(labeled_regions_set)
    #print(len(labeled_regions_set), len(props_lists))
    biggest_r_p = props_lists[0]
    biggest_r_label = props_lists[0].label
    for i in range(0, len(props_lists)):
        pli = props_lists[i]
        if pli.area > biggest_r_p.area:
            biggest_r_p = pli
            biggest_r_label = pli.label
        else:
            pass
    return region_selector(labeled_regions_set, biggest_r_label)    
#from pair of labelled, thresholded images, return an image category (of three)

#from pair of labelled,thresholded images, return the x/y sign or x/y magnitude for the transparent dot


In [4]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/New_setup/*[0-9].avi")#["./tchakamau/New_setup/45681_10Vpp_25.avi", "./tchakamau/New_setup/45701_10Vpp_20.avi"]#glob("./tchakamau/New_setup/*.avi")#  #filenames = 
print(filenames)
num_top_keys = 4
num_full_keys = 7
num_tran_keys = 5
Top_bottoms = np.zeros((len(filenames), num_top_keys))
Full_classif = np.zeros((len(filenames), num_full_keys))
N_transitions = np.zeros((len(filenames), num_tran_keys))
trans_fil = 10

ave = np.average
num_total_frames = 0 
area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)

print("Number files: ", len(filenames))

for vidnum in range(len(filenames)):
    print("Processing vid %s" %vidnum)
    #meta = ffprobe(filenames[vidnum])
    #print(meta)#['@nb_frames'])
    #print(5)
    frames = vreader(filenames[vidnum])
    #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
    #print(frames)
    
    
    
    sides = {"bm":[], "b":[], "t":[], "ucf":[]}
    classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[], "newt":[]}
    transitions = {"->bm" :[], "->br":[], "->bl":[], "->tr":[], "->tl":[]}
    broken_count = 0
    last_class = ''
    last_whole = 0
    filtrate_len = 0
    for fr in frames:
        frame = fr[:, :, 2]
        #print(fr.shape, frame.shape)
        #print(frame)
        num_total_frames += 1
        #print(broken_count, last_class, last_whole, filtrate_len, num_total_frames)
        org = total_threshold_filter(frame, num_total_frames, trans_fil,
                                     broken_count, last_class, last_whole, filtrate_len, 
                                     sides, classes, transitions)
        broken_count, last_class, last_whole, filtrate_len = org
    filtered_len = filtrate_len
    #----------------------------------------------------------------
    trans_len = len([y for x in transitions.values() for y in x])
    skeys = list(sides.keys())
    ckeys = list(classes.keys())
    tkeys = list(transitions.keys())
    slcs = [len(sides[x]) for x in skeys]
    clcs = [len(classes[x]) for x in ckeys]
    trans = [len(transitions[x]) for x in tkeys]
    ucf = len(sides["ucf"])
    newtucf = len(classes["newt"]) + ucf



    Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
    Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
    N_transitions[vidnum,:] = [trans[i] for i in range(len(tkeys))]
    print("finished vid %s" %vidnum)


    
#-----------------------------------------------------------------------




['./tchakamau/New_setup\\45621_10Vpp_22.avi', './tchakamau/New_setup\\45621_10Vpp_23.avi', './tchakamau/New_setup\\45681_10Vpp_24.avi', './tchakamau/New_setup\\45681_10Vpp_25.avi', './tchakamau/New_setup\\45681_10Vpp_26.avi', './tchakamau/New_setup\\45681_10Vpp_27.avi', './tchakamau/New_setup\\45701_10Vpp_18.avi', './tchakamau/New_setup\\45701_10Vpp_19.avi', './tchakamau/New_setup\\45701_10Vpp_20.avi', './tchakamau/New_setup\\45701_10Vpp_21.avi']
Number files:  10
Processing vid 0
finished vid 0
Processing vid 1
finished vid 1
Processing vid 2
finished vid 2
Processing vid 3
finished vid 3
Processing vid 4
finished vid 4
Processing vid 5
finished vid 5
Processing vid 6
finished vid 6
Processing vid 7
finished vid 7
Processing vid 8
finished vid 8
Processing vid 9
finished vid 9


In [12]:
"""Print average values (wuth column labels from slcs, clcs, trans) for rows of the array"""
var = np.var
sums = np.sum
Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
Nt_sums = [sums(N_transitions[:, c]) for c in range(num_tran_keys)]
print(Tb_sums, Fc_sums, Nt_sums)

print("\n Number of frames obbserved: ", num_total_frames)

Tb_percs = [Tb_sums[c] *100 /sum(Tb_sums) for c in range(num_top_keys)]
Fc_percs = [Fc_sums[c] *100/sum(Fc_sums) for c in range(num_full_keys)]
Nt_percs = [Nt_sums[c] *100/sum(Nt_sums) for c in range(num_tran_keys)]

"""Tb_vars = [var(Top_bottoms[:, c]) for c in range(num_top_keys)]
Fc_vars = [var(Full_classif[:, c]) for c in range(num_full_keys)]
Nt_vars = [var(N_transitions[:, c]) for c in range(num_tran_keys)]
print(Nt_vars)"""

print("\nsums in broad classifications")
[print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
print("Total: ", sum(Tb_sums) )
print("\nsums in narrow classifications")
[print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
print("Total: ", sum(Fc_sums ) )
print("\nsums of transitions")
[print(tkeys[i], "sum %.f" %Nt_sums[i]) for i in range(len(tkeys))]
print("Total: ", sum(Nt_sums) )


print("\n%s in broad classifications")
[print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
print("\n%s in narrow classifications")
[print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]
print("\n%s of transitions broad")
print(skeys[0], " %.f%%" %Nt_percs[0],"\n",
       skeys[1], " %.f%%" %(Nt_percs[1] + Nt_percs[2]),"\n",
       skeys[2], " %.f%%" %(Nt_percs[3] + Nt_percs[4]))
print("\n%s of transitions narrow")
[print(tkeys[i], " %.f%%" %Nt_percs[i]) for i in range(len(tkeys))]



print("\nratio of # transitions to time in state (normalized using bm value)")
print(skeys[0], " %.f" %((Nt_sums[0]/Tb_sums[0]) / (Nt_sums[0]/Tb_sums[0])),"\n",
       skeys[1], " %.2f" %((Nt_sums[1] + Nt_sums[2])*Tb_sums[0]/(Nt_sums[0] * Tb_sums[1])),"\n",
       skeys[2], " %.2f" %((Nt_sums[3] + Nt_sums[4]) *Tb_sums[0]/(Nt_sums[0] * Tb_sums[2])))

print("\nratio of # transitions to time in state (normalized using bm value)")
print(ckeys[0], " %.f" %((Nt_sums[0]/Fc_sums[0]) / (Nt_sums[0]/Fc_sums[0])),"\n",
       ckeys[1], " %.2f" %(Nt_sums[1]*Fc_sums[0]/(Nt_sums[0] * Fc_sums[1])),"\n",
       ckeys[2], " %.2f" %(Nt_sums[2]*Fc_sums[0]/(Nt_sums[0] * Fc_sums[2])),"\n",
       ckeys[3], " %.2f" %(Nt_sums[3]*Fc_sums[0]/(Nt_sums[0] * Fc_sums[3])),"\n",
       ckeys[4], " %.2f" %(Nt_sums[4]*Fc_sums[0]/(Nt_sums[0] * Fc_sums[4])))
"""print("\nvariance in %s in broad classifications")
[print(skeys[i], "var %.f%%" %Tb_vars[i]) for i in range(len(skeys))]
print("\nvariance in %s in narrow classifications")
[print(ckeys[i], "var %.f%%" %Fc_vars[i]) for i in range(len(ckeys))]
print("\nvariance in %s of transitions")
[print(tkeys[i], "var %.f%%" %Nt_vars[i]) for i in range(len(tkeys))]"""

[70883.0, 120079.0, 176777.0, 1855.0] [70883.0, 78820.0, 41259.0, 86994.0, 89783.0, 1855.0, 0.0] [241.0, 204.0, 249.0, 187.0, 184.0]

 Number of frames obbserved:  493015

sums in broad classifications
bm sum 70883
b sum 120079
t sum 176777
ucf sum 1855
Total:  369594.0

sums in narrow classifications
bm sum 70883
br sum 78820
bl sum 41259
tr sum 86994
tl sum 89783
ucf sum 1855
newt sum 0
Total:  369594.0

sums of transitions
->bm sum 241
->br sum 204
->bl sum 249
->tr sum 187
->tl sum 184
Total:  1065.0

%s in broad classifications
bm  19%
b  32%
t  48%
ucf  1%

%s in narrow classifications
bm  19%
br  21%
bl  11%
tr  24%
tl  24%
ucf  1%
newt  0%

%s of transitions broad
bm  23% 
 b  43% 
 t  35%

%s of transitions narrow
->bm  23%
->br  19%
->bl  23%
->tr  18%
->tl  17%

ratio of # transitions to time in state (normalized using bm value)
bm  1 
 b  1.11 
 t  0.62

ratio of # transitions to time in state (normalized using bm value)
bm  1 
 br  0.76 
 bl  1.78 
 tr  0.63 
 tl  0.60


'print("\nvariance in %s in broad classifications")\n[print(skeys[i], "var %.f%%" %Tb_vars[i]) for i in range(len(skeys))]\nprint("\nvariance in %s in narrow classifications")\n[print(ckeys[i], "var %.f%%" %Fc_vars[i]) for i in range(len(ckeys))]\nprint("\nvariance in %s of transitions")\n[print(tkeys[i], "var %.f%%" %Nt_vars[i]) for i in range(len(tkeys))]'